<a href="https://colab.research.google.com/github/Amisha1019/assignment/blob/main/task2_nullclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_dim, in_dim // 8, kernel_size=1)
        self.key_conv   = nn.Conv2d(in_dim, in_dim // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_dim, in_dim, kernel_size=1)

        self.gamma = nn.Parameter(torch.zeros(1))

In [ ]:
def forward(self, x):
        batch, C, width, height = x.size()

        # Query, Key, Value
        proj_query = self.query_conv(x).view(batch, -1, width*height).permute(0, 2, 1)  # B x N x C'
        proj_key   = self.key_conv(x).view(batch, -1, width*height)                     # B x C' x N
        energy     = torch.bmm(proj_query, proj_key)                                    # B x N x N
        attention  = F.softmax(energy, dim=-1)                                          # attention map

        proj_value = self.value_conv(x).view(batch, -1, width*height)                   # B x C x N
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))                         # B x C x N
        out = out.view(batch, C, width, height)

        out = self.gamma * out + x   # residual connection
        return out

In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, img_dim, text_dim, hidden_dim):
        super(CrossAttention, self).__init__()
        self.query_conv = nn.Conv2d(img_dim, hidden_dim, kernel_size=1)
        self.key_linear = nn.Linear(text_dim, hidden_dim)
        self.value_linear = nn.Linear(text_dim, hidden_dim)

        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, img_feat, text_emb):
        """
        img_feat: (B, C, W, H)
        text_emb: (B, seq_len, text_dim)
        """
        B, C, W, H = img_feat.size()
        seq_len = text_emb.size(1)


        queries = self.query_conv(img_feat).view(B, -1, W*H).permute(0, 2, 1)


        keys   = self.key_linear(text_emb)
        values = self.value_linear(text_emb)

        energy = torch.bmm(queries, keys.transpose(1, 2))
        attention = F.softmax(energy, dim=-1)

        out = torch.bmm(attention, values)
        out = out.permute(0, 2, 1).contiguous()
        out = out.view(B, -1, W, H)

        return self.gamma * out + img_feat

In [ ]:
def forward(self, img_feat, text_emb):
        """
        img_feat: (B, C, W, H)
        text_emb: (B, seq_len, text_dim)
        """
        B, C, W, H = img_feat.size()
        seq_len = text_emb.size(1)


        queries = self.query_conv(img_feat).view(B, -1, W*H).permute(0, 2, 1)


        keys   = self.key_linear(text_emb)
        values = self.value_linear(text_emb)

In [ ]:
energy = torch.bmm(queries, keys.transpose(1, 2))
attention = F.softmax(energy, dim=-1)

out = torch.bmm(attention, values)
out = out.permute(0, 2, 1).contiguous()
out = out.view(B, -1, W, H)

return self.gamma * out + img_feat

NameError: name 'queries' is not defined

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim=100, text_dim=256):
        super(Generator, self).__init__()
        self.fc = nn.Linear(z_dim, 256*8*8)  # project noise

        self.conv1 = nn.ConvTranspose2d(256, 128, 4, 2, 1)
        self.attn1 = SelfAttention(128)

        self.conv2 = nn.ConvTranspose2d(128, 64, 4, 2, 1)
        self.cross_attn = CrossAttention(64, text_dim, hidden_dim=32)

        self.conv3 = nn.ConvTranspose2d(64, 3, 4, 2, 1)
        self.tanh = nn.Tanh()

    def forward(self, z, text_emb):
        x = self.fc(z).view(-1, 256, 8, 8)
        x = F.relu(self.conv1(x))
        x = self.attn1(x)

        x = F.relu(self.conv2(x))
        x = self.cross_attn(x, text_emb)

        x = self.tanh(self.conv3(x))
        return x

In [ ]:
def forward(self, z, text_emb):
        x = self.fc(z).view(-1, 256, 8, 8)
        x = F.relu(self.conv1(x))
        x = self.attn1(x)

        x = F.relu(self.conv2(x))
        x = self.cross_attn(x, text_emb)

        x = self.tanh(self.conv3(x))
        return x